In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [5]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.ConjugateGradient(model=model, lr=1e-4, line_search_method="const", cg_method="PR")
opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein", cg_method="PR")


all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.36605796217918396
epoch:  1, loss: 0.1960708051919937
epoch:  2, loss: 0.10571490973234177
epoch:  3, loss: 0.0660625472664833
epoch:  4, loss: 0.047632113099098206
epoch:  5, loss: 0.03896179795265198
epoch:  6, loss: 0.034802209585905075
epoch:  7, loss: 0.03277411311864853
epoch:  8, loss: 0.03175155818462372
epoch:  9, loss: 0.031225211918354034
epoch:  10, loss: 0.030953457579016685
epoch:  11, loss: 0.030809607356786728
epoch:  12, loss: 0.030731569975614548
epoch:  13, loss: 0.03068777173757553
epoch:  14, loss: 0.03066111169755459
epoch:  15, loss: 0.030640363693237305
epoch:  16, loss: 0.030640363693237305
epoch:  17, loss: 0.03061521053314209
epoch:  18, loss: 0.030597424134612083
epoch:  19, loss: 0.03058696910738945
epoch:  20, loss: 0.030580561608076096
epoch:  21, loss: 0.03057183511555195
epoch:  22, loss: 0.03057183511555195
epoch:  23, loss: 0.030560895800590515
epoch:  24, loss: 0.030560895800590515
epoch:  25, loss: 0.030542295426130295
epoch:  26,

In [6]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.7252108677464713
Test metrics:  R2 = 0.6898091145881076


In [7]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.ConjugateGradient(model=model, lr=1e-4, line_search_method="const", cg_method="FR")
opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo", cg_method="DY")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=0.1, c2=0.9, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=0.1, c2=0.9, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein", cg_method="FR")

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.8062394857406616
epoch:  1, loss: 0.4199889302253723
epoch:  2, loss: 0.4199889302253723
epoch:  3, loss: 0.2094123512506485
epoch:  4, loss: 0.2094123512506485
epoch:  5, loss: 0.06591888517141342
epoch:  6, loss: 0.028708448633551598
epoch:  7, loss: 0.027985699474811554
epoch:  8, loss: 0.027625033631920815
epoch:  9, loss: 0.027625033631920815
epoch:  10, loss: 0.02762504294514656
epoch:  11, loss: 0.02762504294514656
epoch:  12, loss: 0.02762628346681595
epoch:  13, loss: 0.02745325118303299
epoch:  14, loss: 0.02673564851284027
epoch:  15, loss: 0.02653760462999344
epoch:  16, loss: 0.02653760462999344
epoch:  17, loss: 0.026541614904999733
epoch:  18, loss: 0.02642006427049637
epoch:  19, loss: 0.02642006427049637
epoch:  20, loss: 0.02642008475959301
epoch:  21, loss: 0.026124833151698112
epoch:  22, loss: 0.026124833151698112
epoch:  23, loss: 0.026125632226467133
epoch:  24, loss: 0.026125632226467133
epoch:  25, loss: 0.02153407409787178
epoch:  26, loss: 

In [8]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.7222837551830337
Test metrics:  R2 = 0.7124056969067776
